# BGS SV QA

## Load the Spectro Catalog Database

Start by configuring the database, then load exposures, truth, targets, fiberassign, and the redshift catalog.

In [1]:
import importlib
import desispec.database.redshift
import os
import numpy as np

from   argparse                   import Namespace
import astropy.io.fits            as fits

from   desitarget.targetmask import (desi_mask, mws_mask, bgs_mask)
from   desispec.io.meta import findfile

from   desispec.database.redshift import (setup_db, dbSession, load_file, ObsList, Target, Truth, ZCat, FiberAssign,
                                         load_fiberassign, update_truth)

importlib.reload(desispec.database.redshift)

<module 'desispec.database.redshift' from '/global/common/software/desi/cori/desiconda/20180709-1.2.6-spec/code/desispec/master/py/desispec/database/redshift.py'>

In [2]:
##  
basedir      = '/global/cscratch1/sd/mjwilson/minitest-19.2/spectro/redux/mini/exposures/20200317/00000018'


basedir      = '/global/cscratch1/sd/mjwilson/minitest-19.2/'
targetdir    = '/global/cscratch1/sd/mjwilson/minitest-19.2/targets/'
reduxdir     = '/global/cscratch1/sd/mjwilson/minitest-19.2/spectro/redux/mini/'
expfile      = '/global/cscratch1/sd/mjwilson/minitest-19.2/survey/exposures.fits'
fibassigndir = '/global/cscratch1/sd/mjwilson/minitest-19.2/fiberassign/'

os.environ['DESI_SPECTRO_REDUX'] = os.path.join(basedir, 'spectro', 'redux')
os.environ['DESI_SPECTRO_SIM']   = os.path.join(basedir, 'spectro', 'sim')

os.environ['PIXPROD']            = 'mini'
os.environ['SPECPROD']           = 'mini'

os.environ['DESI_SPECTRO_DATA']  = os.path.join(os.environ['DESI_SPECTRO_SIM'], os.environ['PIXPROD'])

print(os.environ['DESI_SPECTRO_REDUX'])
print(os.environ['DESI_SPECTRO_SIM'])
print(os.environ['DESI_SPECTRO_DATA'])

/global/cscratch1/sd/mjwilson/minitest-19.2/spectro/redux
/global/cscratch1/sd/mjwilson/minitest-19.2/spectro/sim
/global/cscratch1/sd/mjwilson/minitest-19.2/spectro/sim/mini


In [3]:
options      = Namespace(overwrite=True, dbfile=os.path.join(basedir, 'minitest.db'), hostname=None, maxrows=20,
                         chunksize=50000, schema=None, username=None, verbose=False, datapath=basedir)

# We'll be using a SQLite database, ignore the return value of setup_db.
postgresql = setup_db(options)

print('--- Loading exposure list')
load_file(expfile, ObsList, hdu='EXPOSURES', expand={'PASS': 'passnum'})

print('--- Loading truth tables')
load_file(os.path.join(targetdir, 'truth.fits'), Truth, hdu='TRUTH')

for h in ('BGS', 'ELG', 'LRG', 'QSO', 'STAR', 'WD'):
    update_truth(os.path.join(targetdir, 'truth.fits'), 'TRUTH_' + h)

print('--- Loading targets')
load_file(os.path.join(targetdir, 'targets.fits'), Target, hdu='TARGETS',
          expand={'DCHISQ': ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_comp',)})

print('--- loading redshift catalog')
load_file(os.path.join(reduxdir, 'zcatalog-mini.fits'), ZCat, hdu='ZCATALOG',
          expand={'COEFF': ('coeff_0', 'coeff_1', 'coeff_2', 'coeff_3', 'coeff_4',
                            'coeff_5', 'coeff_6', 'coeff_7', 'coeff_8', 'coeff_9',)},
          rowfilter=lambda x: ((x['TARGETID'] != 0) & (x['TARGETID'] != -1)))

print('--- loading fiber assignments')
load_fiberassign(fibassigndir)

print('--- done')

INFO:redshift.py:803:setup_db: Removing file: /global/cscratch1/sd/mjwilson/minitest-19.2/minitest.db.
INFO:redshift.py:812:setup_db: Begin creating tables.
INFO:redshift.py:816:setup_db: Finished creating tables.
--- Loading exposure list
INFO:redshift.py:365:load_file: Read data from /global/cscratch1/sd/mjwilson/minitest-19.2/survey/exposures.fits HDU EXPOSURES
INFO:redshift.py:383:load_file: Integrity check complete on obslist.
INFO:redshift.py:391:load_file: Initial column conversion complete on obslist.
INFO:redshift.py:413:load_file: Column expansion complete on obslist.
INFO:redshift.py:419:load_file: Column conversion complete on obslist.
INFO:redshift.py:426:load_file: Converted columns into rows on obslist.
INFO:redshift.py:433:load_file: Inserted 45 rows in obslist.
--- Loading truth tables
INFO:redshift.py:365:load_file: Read data from /global/cscratch1/sd/mjwilson/minitest-19.2/targets/truth.fits HDU TRUTH
INFO:redshift.py:383:load_file: Integrity check complete on truth.

In [4]:
# Print the table columns and their types.
[(c.name, c.type) for c in ObsList.__table__.columns]

[('expid', Integer()),
 ('tileid', Integer()),
 ('passnum', Integer()),
 ('ra', Float()),
 ('dec', Float()),
 ('ebmv', Float()),
 ('night', String()),
 ('mjd', Float()),
 ('exptime', Float()),
 ('seeing', Float()),
 ('transparency', Float()),
 ('airmass', Float()),
 ('moonfrac', Float()),
 ('moonalt', Float()),
 ('moonsep', Float()),
 ('program', String()),
 ('flavor', String())]

In [5]:
# Print the table columns and their types.
[(c.name, c.type) for c in Target.__table__.columns]

[('release', Integer()),
 ('brickid', Integer()),
 ('brickname', String()),
 ('brick_objid', Integer()),
 ('morphtype', String()),
 ('ra', Float()),
 ('dec', Float()),
 ('ra_ivar', Float()),
 ('dec_ivar', Float()),
 ('dchisq_psf', Float()),
 ('dchisq_rex', Float()),
 ('dchisq_dev', Float()),
 ('dchisq_exp', Float()),
 ('dchisq_comp', Float()),
 ('flux_g', Float()),
 ('flux_r', Float()),
 ('flux_z', Float()),
 ('flux_w1', Float()),
 ('flux_w2', Float()),
 ('flux_ivar_g', Float()),
 ('flux_ivar_r', Float()),
 ('flux_ivar_z', Float()),
 ('flux_ivar_w1', Float()),
 ('flux_ivar_w2', Float()),
 ('mw_transmission_g', Float()),
 ('mw_transmission_r', Float()),
 ('mw_transmission_z', Float()),
 ('mw_transmission_w1', Float()),
 ('mw_transmission_w2', Float()),
 ('nobs_g', Integer()),
 ('nobs_r', Integer()),
 ('nobs_z', Integer()),
 ('fracflux_g', Float()),
 ('fracflux_r', Float()),
 ('fracflux_z', Float()),
 ('fracmasked_g', Float()),
 ('fracmasked_r', Float()),
 ('fracmasked_z', Float()),
 ('f

In [6]:
# Print the table columns and their types.
[(c.name, c.type) for c in FiberAssign.__table__.columns]

[('tileid', Integer()),
 ('fiber', Integer()),
 ('location', Integer()),
 ('numtarget', Integer()),
 ('targetid', BigInteger()),
 ('desi_target', BigInteger()),
 ('bgs_target', BigInteger()),
 ('mws_target', BigInteger()),
 ('target_ra', Float()),
 ('target_dec', Float()),
 ('design_x', Float()),
 ('design_y', Float()),
 ('brickname', String()),
 ('fiberstatus', Integer()),
 ('design_q', Float()),
 ('design_s', Float()),
 ('lambda_ref', Float()),
 ('objtype', String()),
 ('petal_loc', Integer()),
 ('device_loc', Integer()),
 ('priority', Integer()),
 ('subpriority', Float())]

In [4]:
#.filter(Target.desi_target.op('&')(desi_mask.BGS_ANY) != 0)\
#                   .filter(FiberAssign.tileid == ObsList.tileid)\
# , ObsList.expid, ObsList.tileid, ObsList.night, ObsList.mjd

'''
q = dbSession.query(Target.targetid, FiberAssign.tileid, FiberAssign.fiber, ObsList.expid, ObsList.mjd, ObsList.night)\
                   .filter(Target.targetid    == FiberAssign.targetid)\
                   .filter(FiberAssign.tileid == ObsList.tileid)\
                   .order_by(ObsList.expid, FiberAssign.fiber)\
                   .all()

targetid, tileid, fiberid, expid, mjd, night = zip(*q)

for x in [targetid, tileid, fiberid, expid, mjd, night]:
    x = np.array(x)
'''
q = dbSession.query(ZCat.targetid, Truth.truez, ZCat.z, ZCat.zwarn, Target.fiberflux_r, ((ZCat.z - Truth.truez)/(1.0 + Truth.truez)).label('dz'))\
                   .filter(Truth.targetid     == ZCat.targetid)\
                   .filter(Target.targetid    == ZCat.targetid)\
                   .filter(ZCat.targetid      == FiberAssign.targetid)\
                   .filter(FiberAssign.tileid == ObsList.tileid)\
                   .filter(Target.desi_target.op('&')(desi_mask.BGS_ANY) != 0)\
                   .all()

UnboundExecutionError: Could not locate a bind configured on mapper Mapper|ZCat|zcat, SQL expression or this Session

In [57]:
print(np.unique(expid, return_counts=True))

(array([ 6,  7,  8,  9, 10, 11, 18, 19, 20, 21, 28, 29, 30, 31, 32, 39, 40,
       41, 42, 43, 44]), array([4594, 4594, 4594, 4594, 4594, 4594, 4595, 4595, 4595, 4595, 4594,
       4594, 4594, 4593, 4593, 4591, 4591, 4586, 4586, 4590, 4590]))


In [ ]:
specids = []

##  Sorted by exposure id. 
for exp in expid:    
    fmap_path = findfile('fibermap', night=night[i], expid=exp)

    print('Loading %s' % fmap_path)
    
    fmap      = fits.open(fmap_path)
    
    ##  Spectrograph id for each fiber in the exposure.  
    specids.append(fmap[1].data['SPECTROID'])
    
    fmap.close()

In [43]:
findfile('fibermap', night=night[0], expid=expid[0])

'/global/cscratch1/sd/mjwilson/minitest-19.2/spectro/sim/mini/20200411/00000043/fibermap-00000043.fits'

In [ ]:
cframes = []

for i in range(10):
    cframes.append(findfile('cframe', night=night[i], expid=expid[i], nside=64, camera='r' ))
    
    print(targetid[i], tileid[i], expid[i], mjd[i], night[i], files[i])